In [1]:
import pandas as pd
import numpy as np

In [2]:
dfcatalogo = pd.read_csv(r'C:\Users\oomph\Documents\Henry Data Science\DTS09_Proyecto_Individual\df_catalogo_v2.csv')


In [8]:
dfcatalogo.sample(10)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,plataforma,movieId,duration_int,duration_type,avg_rating
10839,s1172,movie,roving mars,george butler,"paul newman, steve squyres, rob manning, dr. e...",united states,2019-11-12,2006,g,41 min,"documentary, family",explore the surface of the red planet through ...,disney+,ds1172,41.0,min,3.559395
1408,s1409,tv show,orphan black (4k uhd),NaN,"tatiana maslany, jordan gavaris, maria doyle k...",NaN,NaN,2014,tv-ma,2 seasons,"drama, science fiction",season two hits the ground running with sarah ...,amazon,as1409,2.0,seasons,3.448898
14517,s327,movie,beethoven's 2nd,rod daniel,"charles grodin, bonnie hunt, nicholle tom, chr...",united states,2021-08-01,1993,pg,88 min,"children & family movies, comedies",after st. bernard beethoven sires a brood with...,netflix,ns327,88.0,min,3.597192
10186,s519,movie,mickey and the seal,charles nichols,NaN,united states,2020-05-29,1948,tv-g,7 min,"animation, family",mickey and pluto get into trouble with a misch...,disney+,ds519,7.0,min,3.565173
4347,s4348,movie,rambo: last blood (4k uhd),adrian grunberg,"sylvester stallone, paz vega, sergio peris-men...",NaN,NaN,2019,r,89 min,"action, adventure","in a deadly journey of vengeance, rambo must c...",amazon,as4348,89.0,min,3.518889
13133,s2016,tv show,the boardroom,NaN,NaN,NaN,2019-10-30,2019,g,2 seasons,documentaries,fans get an inside look at the business of spo...,hulu,hs2016,2.0,seasons,3.567164
2759,s2760,movie,criminally insane,nick millard,priscilla alden,NaN,NaN,2007,18+,62 min,drama,an obese woman recently released from an insan...,amazon,as2760,62.0,min,3.537736
15424,s1234,movie,fate of alakada,kayode kasum,"toyin abraham, broda shaggi, calabar chic, mun...",nigeria,2021-03-05,2020,tv-14,113 min,"comedies, international movies","faking her way through any situation, a social...",netflix,ns1234,113.0,min,3.559375
22830,s8640,movie,tukaram,chandrakant kulkarni,"jitendra joshi, prateeksha lonkar, sharad ponk...",india,2018-02-15,2012,tv-pg,162 min,"dramas, faith & spirituality, international mo...",a child whose innocence and devotion set him a...,netflix,ns8640,162.0,min,3.493865
15272,s1082,tv show,"nicky, ricky, dicky & dawn",NaN,"brian stepanek, allison munn, aidan gallagher,...",united states,2021-04-12,2018,tv-g,4 seasons,"kids' tv, tv comedies","just because they're quadruplets, that doesn't...",netflix,ns1082,4.0,seasons,3.568565


funcion get max duration

In [4]:
def get_max_duration(year: int, platform: str, duration_type: str) -> dict:
    # Filtrar el dataframe por tipo de contenido, año y plataforma
    filtered_df = dfcatalogo[(dfcatalogo['type'] == 'movie') & 
                            (dfcatalogo['release_year'] == year) & 
                            (dfcatalogo['plataforma'] == platform)]
    
    # Obtener la película con la duración máxima en el tipo especificado
    max_duration = filtered_df[filtered_df['duration_type'] == duration_type]['duration_int'].max()
    max_duration_movie = filtered_df[(filtered_df['duration_int'] == max_duration) &
                                     (filtered_df['duration_type'] == duration_type)]['title'].values
    
    # Verificar que el array de resultados no está vacío antes de intentar acceder a su primer elemento
    if len(max_duration_movie) > 0:
        return {'pelicula': max_duration_movie[0]}
    else:
        return {'pelicula': ''}

In [5]:
# Ejemplo de llamado a la función get_max_duration
year = 2016
platform = 'amazon'
duration_type = 'min'

max_duration_movie = get_max_duration(year, platform, duration_type)
print(f'La película con duración máxima en el año {year} en la plataforma {platform} en {duration_type} es: {max_duration_movie}')

La película con duración máxima en el año 2016 en la plataforma amazon en min es: {'pelicula': 'gentle evening rain'}


funcion get score count

In [6]:
def get_score_count(df, plataforma, scored, anio):
    # Seleccionar solo las películas de la plataforma y el año especificado
    peliculas = df[(df['plataforma'] == plataforma) &
                   (df['type'] == 'movie') &
                   (df['release_year'] == anio)]
    # Filtrar solo las películas con un puntaje mayor al especificado
    peliculas_con_puntaje = peliculas[peliculas['avg_rating'] > scored]
    # Obtener la cantidad de películas que cumplen los criterios
    cantidad = len(peliculas_con_puntaje)
    return {
        'plataforma': plataforma,
        'cantidad': cantidad,
        'anio': anio,
        'score': scored
    }

In [7]:
get_score_count(dfcatalogo, 'disney+', 1.45, 2014)

{'plataforma': 'disney+', 'cantidad': 34, 'anio': 2014, 'score': 1.45}

funcion get count platform

In [8]:
def get_count_platform(df, plataforma):
    # Seleccionar solo las películas de la plataforma especificada
    peliculas = df[(df['plataforma'] == plataforma) &
                   (df['type'] == 'movie')]
    # Obtener la cantidad de películas que cumplen los criterios
    cantidad = len(peliculas)
    return {
        'plataforma': plataforma,
        'peliculas': cantidad
    }

In [9]:
get_count_platform(dfcatalogo, 'hulu')

{'plataforma': 'hulu', 'peliculas': 1484}

funcion get actor

In [9]:
def get_actor(platform:str, year:int ):
    df_filtered = dfcatalogo[(dfcatalogo['plataforma'] == platform) & (dfcatalogo['release_year'] == year)]
    actor_count = {}
    for actors in df_filtered['cast']:
        if isinstance(actors, str):
            for actor in actors.split(','):
                actor_count[actor.strip()] = actor_count.get(actor.strip(), 0) + 1
    if len(actor_count) == 0:
        return "No se encontraron actores"
    else:
        return 'Actor que más se repite según plataforma y año:  ' + max(actor_count, key=actor_count.get)

In [17]:
respuesta = get_actor('amazon', 2010)
print(respuesta)

Actor que más se repite según plataforma y año:  cassandra peterson


funcion prod per county

In [14]:
def prod_per_county(tipo: str, pais: str, anio: int):
    # Filtramos el DataFrame según el tipo de contenido, el país y el año
    df_filtered = dfcatalogo[(dfcatalogo['type'] == tipo) & (dfcatalogo['country'] == pais) & (dfcatalogo['release_year'] == anio)]

    # Contamos el número total de elementos del DataFrame filtrado
    total_peliculas = df_filtered.shape[0]

    return {'pais': pais, 
            'anio': anio, 
            'total de contenido': total_peliculas,
            'tipo_contenido': tipo}

In [15]:
respuesta = prod_per_county('tv show', 'france', 2020,)
print(respuesta)

{'pais': 'france', 'anio': 2020, 'total de contenido': 8, 'tipo_contenido': 'tv show'}


funcion get contents

In [16]:
def get_contents(rating: str):
    # Utilizamos value_counts para contar el número de contenidos según el rating de audiencia
    contenido = dfcatalogo['rating'].value_counts().get(rating, 0)

    return {'rating': rating, 'contenido': contenido}

In [17]:
respuesta = get_contents('r')
print(respuesta)

{'rating': 'r', 'contenido': 2154}


In [21]:
dfcatalogo.sample(10)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,plataforma,movieId,duration_int,duration_type,avg_rating
5280,s5281,movie,hum tum,kunal kohli,"kiron kher, rishi kapoor, saif ali khan, rani ...",NaN,NaN,2004,all,143 min,"comedy, international, romance",karan (saif ali khan) is a cartoonist. his cha...,amazon,as5281,143.0,min,3.510891
17718,s3528,tv show,"the mind, explained",NaN,emma stone,united states,2019-09-12,2019,tv-14,1 season,"docuseries, science & nature tv",ever wonder what's happening inside your head?...,netflix,ns3528,1.0,season,3.523622
5361,s5362,movie,asterix at the olympic games,"frédéric forestier, thomas langmann","gérard depardieu, clovis cornillac, benoît poe...",NaN,NaN,2008,13+,117 min,"action, comedy",astérix and obélix compete at the olympics in ...,amazon,as5362,117.0,min,3.507384
1913,s1914,tv show,laakhon mein ek (tamil),NaN,"shweta tripathi sharma, sandeep mehta, suyash ...",NaN,NaN,2019,tv-nr,2 seasons,"comedy, drama",dr shreya is posted in sitlapur village to con...,amazon,as1914,2.0,seasons,3.592751
7581,s7582,movie,anacondas: trail of blood,don e. fauntleroy,"crystal allen, linden ashby, danny midwinter, ...",NaN,NaN,2006,r,89 min,"action, horror, suspense",even though the two monster snakes from anacon...,amazon,as7582,89.0,min,3.533186
5560,s5561,movie,split second,tony maylam,"rutger hauer, kim catrall, michael j. pollard,...",united kingdom,NaN,1992,r,91 min,"horror, science fiction","in a flooded dystopian future, detective harle...",amazon,as5561,91.0,min,3.462882
14482,s292,tv show,shaman king,NaN,"yoko hikasa, katsuyuki konishi, megumi hayashi...",japan,2021-08-09,2021,tv-14,1 season,"anime series, international tv shows",medium yoh asakura enters a battle tournament ...,netflix,ns292,1.0,season,3.544492
4424,s4425,tv show,word of honor,NaN,"zhehan zhang, jun gong, ye zhou, wenyuan ma, x...",NaN,2021-04-17,2021,18+,2 seasons,"action, drama, romance",(starting from ep 19 and after) zishu zhou (zh...,amazon,as4425,2.0,seasons,3.526998
7934,s7935,movie,the witch of kings cross,sonia bible,NaN,australia,NaN,2020,16+,75 min,documentary,"allegations of satanic rituals, obscene art an...",amazon,as7935,75.0,min,3.612069
15223,s1033,tv show,love naggers,NaN,"seo jang-hoon, kim sook, han hye-jin, kwak jun...",south korea,2021-04-16,2021,tv-14,1 season,"international tv shows, stand-up comedy & talk...","from the quirky to the scandalous, any relatio...",netflix,ns1033,1.0,season,3.550881
